In [ ]:
import sys, pathlib, zipfile
from pathlib import Path

HERE = pathlib.Path().resolve()
REPO = HERE  # assuming this notebook lives in the repo root
sys.path.insert(0, str(REPO))

DATA    = REPO / "data"
OUTPUT  = REPO / "outputs"

for p in (OUTPUT):
    p.mkdir(exist_ok=True, parents=True)

# 3) find & unzip every `train_*.zip` exactly once
zip_files = sorted(DATA.glob("train_*.zip"))
for zip_path in zip_files:
    extract_to = DATA / zip_path.stem 
    if not extract_to.exists():
        print(f">>> Extracting {zip_path.name} → {extract_to}/")
        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(extract_to)
    else:
        print(f"Already extracted: {extract_to}/")

In [ ]:
import sys
from pathlib import Path

HERE = Path().resolve() 
REPO = HERE                      
sys.path.insert(0, str(REPO))

DATA       = REPO / "data"
OUTPUT     = REPO / "outputs" / "VGG16_subset_contrast_100"
IMG_SUBSET = DATA / "train_contrast_100_subset"  # change to folder names of unzipped files

OUTPUT.mkdir(parents=True, exist_ok=True)

import torch
from torchvision import transforms
from torchvision.models import vgg16, VGG16_Weights
import numpy as np

from functions.network_data2 import NetworkData
from functions.image           import ImageDataset
import interface_DeepFramework.DeepFramework as DeepF

def preproces_imagenet_img(imgs_hr):
    img = np.array(imgs_hr)
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
           mean=[0.485,0.456,0.406],
           std =[0.229,0.224,0.225]
        )
    ])
    return [transform(img)]


def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model  = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
    model.to(device).eval()
    for name, module in model.named_modules():
        module.auto_name = name

    deepmodel = DeepF.deep_model(model)

    layers_interest = [
      ["features.0", 0],  # conv1_1
      ["features.2", 0],  # conv1_2
      ["features.5", 0],  # conv2_1
      ["features.7", 0],  # conv2_2
    ]

    Path_images = str(IMG_SUBSET)

    dataset = ImageDataset(
      src_dataset           = Path_images,
      target_size           = (224,224),
      preprocessing_function= preproces_imagenet_img,
      color_mode            = "rgb"
    )

    Nefesimodel = NetworkData(
      model            = deepmodel,
      layer_data       = layers_interest,
      save_path        = str(OUTPUT),
      dataset          = dataset,
      default_file_name= "VGG16_subset_contrast_100",
      input_shape      = [(1,3,224,224)]
    )

    Nefesimodel.generate_neuron_data()

    # 7) activation calculus
    Nefesimodel.eval_network()
    print("Activation Calculus done!")
    Nefesimodel.save_to_disk("activations")

    # 8) neuron features
    Nefesimodel.calculateNF()
    print("NF done!")
    Nefesimodel.save_to_disk("withNF")

    # 9) color‐selectivity index
    for layer in Nefesimodel.get_layers_name():
        layer_data = Nefesimodel.get_layer_by_name(layer)
        print("Computing CSI on", layer)
        for idx in range(Nefesimodel.get_len_neurons_of_layer(layer)):
            neuron = Nefesimodel.get_neuron_of_layer(layer, idx)
            neuron.color_selectivity_idx_new(
              Nefesimodel, layer_data, dataset
            )
    Nefesimodel.save_to_disk("Normal_class")
    print("Saved object with CSI included")

if __name__ == "__main__":
    main()

In [ ]:
# ─── export_nfs_and_metrics.py ────────────────────────────────────────────────
import sys
from pathlib import Path
import math
import pandas as pd
import matplotlib.pyplot as plt

# 1) Make sure we can import your nefesi code in functions/
HERE = Path().resolve()          # if this file/notebook sits at the repo root
REPO = HERE                      # else HERE.parent if it's in a subfolder
sys.path.insert(0, str(REPO))

# 2) define all the relative paths
DATA      = REPO / "data"                      # where your train_*_subset/ folders live
OUTPUT    = REPO / "outputs" / "VGG16_subset_contrast_100"
OBJ_FINAL = OUTPUT / "Normal_class.obj"
NF_DIR    = OUTPUT / "nf_grids"
CSV_OUT   = OUTPUT / "metrics_all.csv"

# 3) layers of interest
LAYERS = ["features.0", "features.2", "features.5", "features.7"]

# 4) ensure output dirs exist
NF_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT.mkdir(parents=True, exist_ok=True)

# 5) import nefesi machinery
from functions.network_data2 import NetworkData

def normalize_patch(patch):
    """Scale an RGB patch to [0,1]."""
    mn, mx = patch.min(), patch.max()
    return (patch - mn) / (mx - mn) if mx > mn else patch - mn

def main():
    # load the fully-populated NetworkData object
    nd = NetworkData.load_from_disk(str(OBJ_FINAL))

    records = []

    # loop through each layer
    for layer in LAYERS:
        n_neurons = nd.get_len_neurons_of_layer(layer)
        cols      = int(math.ceil(math.sqrt(n_neurons)))
        rows      = int(math.ceil(n_neurons / cols))

        # 1) Make the NF grid
        fig, axes = plt.subplots(rows, cols,
                                 figsize=(cols*1.5, rows*1.5),
                                 squeeze=False)
        for idx in range(rows*cols):
            r, c = divmod(idx, cols)
            ax   = axes[r][c]
            ax.axis("off")
            if idx < n_neurons:
                neuron = nd.get_neuron_of_layer(layer, idx)
                patch  = neuron._neuron_feature
                ax.imshow(normalize_patch(patch))
                ax.set_title(str(idx), fontsize=6)

        fig.suptitle(f"{layer} — all {n_neurons} neurons", fontsize=12)
        plt.tight_layout(rect=[0,0,1,0.95])
        out_png = NF_DIR / f"{layer.replace('.','_')}_all_nfs.png"
        fig.savefig(str(out_png), dpi=150)
        plt.close(fig)
        print("Saved full NF grid →", out_png)

        # 2) collect metrics for CSV
        for idx in range(n_neurons):
            neuron  = nd.get_neuron_of_layer(layer, idx)
            max_act = float(neuron.activations[0])
            # call the CSI method to get the value
            csi_val = neuron.color_selectivity_idx_new(nd,
                                                       nd.get_layer_by_name(layer),
                                                       nd.dataset)
            records.append({
                "layer":         layer,
                "neuron_idx":    idx,
                "max_activation": max_act,
                "csi":           csi_val
            })

    # 3) write out the combined CSV
    df = pd.DataFrame(records)
    df.to_csv(str(CSV_OUT), index=False)
    print(f"Wrote metrics CSV → {CSV_OUT} ({len(df)} rows)")

if __name__ == "__main__":
    main()